In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_core.documents import Document

documents= [
    Document(page_content="Ronaldo is a great finisher, he have won multiple Champions League trophies for his team", metadata={"source": "ucl"}),
    Document(page_content="Messi is a great dribbler, he won the 2022 Fifa worldcup", metadata={"source": "la liga"}),
    Document(page_content="Neymar is a great playmaker, his debut for Barcelona was a pure entertainment", metadata={"source": "ligue 1"}),
    Document(page_content="Mbappe is a great sprinter, he's also known by the common Ninja turtles meme", metadata={"source": "ligue 1"}),
]

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

In [ ]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    documents,embedding=embeddings
)

In [ ]:
vectorstore

In [ ]:
vectorstore.similarity_search("best debut")

In [ ]:
await vectorstore.asimilarity_search("best debut")

In [ ]:
vectorstore.similarity_search_with_score("best finisher")

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# With the help of RunnableLambda we are able to create a retriever for the vectorstor
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

In [ ]:
retriever.batch(["best dribbler","best finisher"])

In [ ]:
# Another way

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.batch(["best dribbler","best finisher"])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """Answer this question based on the provided context only
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("human",message)
    ]
)
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
# RunnablePassthrough is used to pass the parameter using invoke
rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

In [ ]:
response = rag_chain.invoke("who is the best finisher")
response.content